<a href="https://colab.research.google.com/github/Behnam9473/AI/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Import Necessary Libraries

First, we need to import the necessary libraries.



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Step 2: Load and Preprocess Data

Next, let's load and preprocess the data. We will use Keras' ImageDataGenerator to load images from directories.




In [ ]:
# Set paths to the data directories
train_dir = '/path/to/train'
validation_dir = '/path/to/validation'

# Create ImageDataGenerators for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Step 3: Load Pre-trained VGG16 Model

We will load the VGG16 model without the top (fully connected) layers.

In [ ]:
# Load the VGG16 model, excluding the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


Step 4: Add Custom Layers

We will add our own fully connected layers on top of the VGG16 base model.

In [ ]:
# Flatten the output of the base model
x = Flatten()(base_model.output)

# Add a fully connected layer with 256 units and ReLU activation
x = Dense(256, activation='relu')(x)

# Add a dropout layer to prevent overfitting
x = tf.keras.layers.Dropout(0.5)(x)

# Add a final output layer with a single unit and sigmoid activation
output = Dense(1, activation='sigmoid')(x)

# Combine the base model and the custom layers into a new model
model = Model(inputs=base_model.input, outputs=output)


Step 5: Freeze the Base Model Layers

We will freeze the layers of the base model to prevent them from being updated during training.

In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False


Step 6: Compile the Model

Next, we will compile the model with a binary cross-entropy loss function and the Adam optimizer.

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


Step 7: Train the Model

We will train the model using the training and validation generators.

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Step 8: Fine-Tune the Model

Optionally, we can fine-tune the model by unfreezing some of the layers of the base model and training the whole network with a lower learning rate.

In [ ]:
# Unfreeze some layers of the base model
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compile the model again with a lower learning rate
model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model again
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)
